In [1]:
from torch.utils.data import DataLoader
from datasets import dataset
import argparse
from model.model import *

C:\Users\hoang\anaconda3\envs\ComputerVision\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hehe= dataset.FaceImageFolderDataset(root= args.folderdataset_dir)

NameError: name 'args' is not defined

In [ ]:
la=10
ua=110
lm=0.45
um=0.8
lg=35

# settings
MODEL_ARC='alexnet'
OUTPUT='test'

In [ ]:
python -u trainer.py --backbone alexnet --folderdataset_dir datasets/raw --pretrained_backbone True --workers 1 --epochs 25 --start-epoch 0 --batch-size 1 --embedding-size 512 --last-fc-size 2 --print-freq 2 --pth-save-fold test --pth-save-epoch 1 

In [59]:
python -u trainer.py --backbone {MODEL_ARC} --folderdataset_dir datasets/raw --pretrained_backbone True --workers 1 --epochs 25 --start-epoch 0 --batch-size 1 --embedding-size 512 --last-fc-size 2 --print-freq 2 --pth-save-fold {OUTPUT} --pth-save-epoch 1 

=> parse the args ...
{'arc_scale': 64,
 'backbone': 'alexnet',
 'batch_size': 1,
 'embedding_size': 512,
 'epochs': 25,
 'folderdataset_dir': 'datasets/raw',
 'l_a': 10,
 'l_margin': 0.45,
 'lambda_g': 20,
 'last_fc_size': 2,
 'lr': 0.1,
 'lr_drop_epoch': [30, 60, 90],
 'lr_drop_ratio': 0.1,
 'momentum': 0.9,
 'pretrained_backbone': 'True',
 'print_freq': 2,
 'pth_save_epoch': 1,
 'pth_save_fold': 'test',
 'start_epoch': 0,
 'train_list': '',
 'u_a': 110,
 'u_margin': 0.8,
 'vis_mag': 1,
 'weight_decay': 0.0001,
 'workers': 1}
min lambda g is 22.586666666666673, currrent lambda is 20
=> torch version : 1.12.1
=> ngpus : 1
=> modeling the network ...
=> building the oprimizer ...
SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)
=> building the dataloader ...


Traceback (most recent call last):
  File "T:\Git_repo\Face_Recognition\face_embedding\trainer.py", line 265, in <module>
    main(args)
  File "T:\Git_repo\Face_Recognition\face_embedding\trainer.py", line 110, in main
    main_worker(ngpus_per_node, args)
  File "T:\Git_repo\Face_Recognition\face_embedding\trainer.py", line 134, in main_worker
    train_loader = dataset.train_loader(args)
  File "T:\Git_repo\Face_Recognition\face_embedding\datasets\dataset.py", line 119, in train_loader
    print(hehe)
NameError: name 'hehe' is not defined


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--backbone', default='iresnet18', type=str,
                    help='backbone architechture')
parser.add_argument('--pretrained_backbone', default=False, type=str,
                    help='Use pretrain backbone')
parser.add_argument('--resume', default=None, type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--folderdataset_dir', default='datasets/raw', type=str,
                    help='Path to Face Image Folder Dataset')
parser.add_argument('--feat_list', default='datasets/face_embdding.txt', type=str,
                    help='Path for saveing features file')
parser.add_argument('-j', '--workers', default=2, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('-b', '--batch-size', default=512, type=int, metavar='N',
                    help='mini-batch size (default: 256), this is the total '
                    'batch size of all GPUs on the current node when '
                    'using Data Parallel or Distributed Data Parallel')
parser.add_argument('--embedding-size', default=512, type=int,
                    help='The embedding feature size')
parser.add_argument('--cpu-mode', action='store_true', help='Use the CPU.')

args=parser.parse_args(''.split())


In [4]:
model= build_backbone(args)

In [9]:
if not args.cpu_mode:
    model = model.cuda()
    device="cuda"
else:
    device="cpu"
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((112, 112)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0., 0., 0.],
                                     std=[1., 1., 1.]),
])    

data_inf = dataset.FaceImageFolderDataset(root= args.folderdataset_dir, transform = transform)
dataloader_inf=DataLoader(data_inf, 
                          batch_size= args.batch_size, 
                          num_workers= args.workers,
                          pin_memory=True,
                          shuffle=False,
)
cprint('=> starting face embdding...', 'green')
cprint('=> embdding feature will be saved into {}'.format(args.feat_list))
model.eval()

file= open(args.feat_list, 'w')

with torch.no_grad():
    for i, (imgs, labels) in enumerate(dataloader_inf):
        imgs=imgs.to(device)
        embedding_feats = model(imgs)
        embedding_feats = embedding_feats.data.cpu().numpy()
        
        for feat, label in zip(embedding_feats, labels):
            file.write('{} '.format(label))
            for r in feat:
                file.write('{} '.format(r))
            file.write('\n')
    file.close()

=> starting face embdding...
=> embdding feature will be saved into datasets/face_embdding.txt
